# UTMB Average Pace Predictions

To make predictions and pacing plan recommendations, a case base reasoning approach was used.
This approach to ultra marathon prediction uses the experience of a runner and the experience
of similar runners to predict their PB average pace and recommend a pacing plan. The non PB
and PB of every runner of the UTMB who has ran two or more races is found. Each record contains the
individual stage paces and average pace of the non PB run and the PB run. These times are weighted according to their similarity to the query
runner (most similar gets a higher weight), to see if better predictions can be made.


Using this case base of 10 similar runners, a challenging but achievable average pace can be
produced. Thus far, the 10 most similar non PB average paces to the query runners non PB
average pace has been highlighted. The case base contains both the non PB and PB average
pace of the most similar runners. The PB average pace of these runners is then highlighted. The
runner in the case base with the fastest average pace is identified. The reason for this is that this
Page 29 of 41
would output the best achievable average pace possible for each runner. To make the prediction
relative the average pace is weighted by comparing the non PB of both the query runner and
fastest case. For example, if the fastest case has a non PB of 11 minutes per kilometres, and
the query has 11.5 minutes per kilometre, the query pace is divided by the fastest case pace, and
fastest case PB is multiplied by the value. This means it caters for the fact that the similar non
PB average pace could be slightly slower or faster than the query non PB average pace. The
second type of prediction made is the mean of the PB average paces of the similar runners. Each
average pace is aggregated and divided the number of runners. This should be 10 but may be
lower for runners who do not have 10 similar runners of equal category. As previously stated,
cosine and euclidean distance are used to find the most similar runners to the query runner.
These distances were used to weight the most similar heavier than the least similar runners. For
example, the most similar runner could have a cosine similarity of 0.997 to the query runner. To
get the weight this figure is divided by the sum of all cosine similarities, and then multiplied by
the average pace. The sum of the average paces multiplied by their weights are then found. This
process is carried out for both euclidean distance values as well as cosine distance values. This
results in four predictions in total: Fastest case PB, Mean of cases PB, euclidean weighted mean
of cases PB, and cosine weighted mean of cases PB.


In [1]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

I aim to produce a challenging but not impossible average pace prediction for each runner. I will first get the runner's non personal best average pace and find the most similar non personal best average pace to that runner. I will then find the personal best average pace of these runners. By average these pb times, or getting the best personal best average pace of these runners, I will be able to produce a predicted average pace. 

In [91]:
# Constants
pace_threshold = 1
#euclidean_distance_threshold = 200

In [92]:
# Read in Split Timings
df = pd.read_csv('../../data/1km_splits_full.csv', encoding = "ISO-8859-1")

In [93]:
len(df)

19579

In [94]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'bib','team', 'rank', 'nationality'], axis = 1)

In [95]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Male,Senior Men (23-39),5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Senior Men (23-39),5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2017,0d928f7482046900e19dba217d7befa5,Male,Senior Men (23-39),5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,2017,93f20063908ba5d48724dcf31740a52a,Male,Senior Men (23-39),5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,2017,7479fb90beffad048c5f772ba1915b4e,Male,Senior Men (23-39),5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


In [96]:
df.shape

(19579, 46)

In [97]:
df.groupby('category').count()

,Year,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),7881,7881,7881,5994,560,954,5990,801,7749,7435,7379,7389,7116,7098,6886,6834,6869,1697,816,5475,6215,6176,5779,5764,5339,1816,540,2726,1811,534,2722,4254,714,4262,714,3707,524,712,3705,523,711,4408,4931,4930,7877
Masters Men 2 (50-59),3393,3393,3393,2592,220,376,2588,373,3312,3145,3116,3119,2966,2944,2848,2814,2838,675,330,2172,2462,2438,2230,2220,2004,736,183,974,733,183,970,1608,239,1606,239,1403,173,235,1398,172,236,1633,1805,1805,3388
Masters Men 3 (60-69),520,520,520,371,43,61,370,74,502,458,453,454,423,417,388,387,384,70,35,281,309,304,261,254,216,100,23,76,100,23,77,180,13,178,13,148,22,13,148,21,13,162,183,183,520
Masters Men 4 (70-79),29,29,29,23,1,4,23,2,28,27,25,26,23,23,21,21,20,2,1,14,15,15,12,12,7,3,0,2,3,0,2,5,0,4,0,4,0,0,4,0,0,4,4,4,29
Masters Women 1 (40-49),719,719,719,560,51,69,560,81,702,668,660,665,633,625,607,601,602,166,72,466,537,530,474,473,431,140,29,240,140,29,239,341,61,343,61,309,27,61,310,27,61,370,397,397,717
Masters Women 2 (50-59),287,287,287,220,15,37,220,26,278,261,259,258,239,237,227,228,224,51,23,171,191,187,169,168,150,60,12,69,60,12,69,117,20,116,20,100,12,20,101,12,20,121,133,133,285
Masters Women 3 (60-69),26,26,26,21,0,3,21,2,24,21,22,21,18,18,17,16,16,8,5,7,12,12,9,9,8,3,1,4,3,1,4,8,0,8,0,7,1,0,7,1,0,7,8,8,26
Senior Men (23-39),6042,6042,6042,4555,501,700,4555,674,5943,5737,5703,5706,5511,5515,5366,5315,5374,1325,639,4360,4955,4917,4701,4687,4354,1493,367,2247,1481,355,2248,3418,586,3423,586,3046,344,580,3044,344,580,3628,3972,3967,6036
Senior Women (23-39),646,646,646,519,31,69,518,51,630,614,606,607,585,579,561,558,563,156,70,440,498,493,460,457,421,134,33,237,134,33,234,326,66,324,66,284,32,66,285,32,66,348,380,380,645


In [99]:
df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
2471,2017,6edbc123d2c45fa61b34a1f47eae4a2c,Female,Masters Women 2 (50-59),10.267157,NaN,NaN,9.618421,NaN,11.441748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.442442
2485,2017,dd9e346cc77bdb575a062caf1e045922,Female,Masters Women 2 (50-59),11.279412,NaN,NaN,10.745614,NaN,12.241100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.422042
2494,2017,086c45154e617591ad9887ce3358be2d,Female,Masters Women 2 (50-59),10.833333,NaN,NaN,11.392544,NaN,13.122977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.782952
2595,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),7.949755,NaN,NaN,5.844298,NaN,8.763754,9.913580,13.748485,7.372549,10.580906,14.908163,13.258333,7.566667,8.939024,18.055556,NaN,14.758503,8.916667,8.650641,15.911111,6.843333,8.625000,NaN,NaN,11.520649,NaN,NaN,8.564626,16.606918,NaN,8.194444,NaN,17.140693,NaN,NaN,14.343434,NaN,NaN,8.497748,1753.933333,433.900000,11.018994
5066,2016,518c744fb20eb7d4c43afd66c1410b57,Female,Masters Women 2 (50-59),10.808824,NaN,NaN,8.978070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.893447
5079,2016,8cb28bf1befd988376fe7152453f08e9,Female,Masters Women 2 (50-59),11.164216,NaN,NaN,11.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.207108
5081,2016,85d4d22c35b487dcb6fa7c4fcfca9ae3,Female,Masters Women 2 (50-59),13.436275,NaN,NaN,9.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.259804
7405,2015,cbf15f23ef5ead61ed9491f0e92aa9b8,Female,Masters Women 2 (50-59),9.091912,NaN,NaN,7.708333,NaN,9.885113,14.232510,18.296970,9.911765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.521101
7546,2015,ea16550cfbbfd6cd4327356dc80126e6,Female,Masters Women 2 (50-59),9.000000,NaN,NaN,9.063596,NaN,11.915858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.993151
7559,2015,7c2c1575d30df2b22d3662dc49bc41bb,Female,Masters Women 2 (50-59),10.167892,NaN,NaN,8.837719,NaN,11.711974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.239195


In [101]:
df.loc[1613]

Year                                                        2017
name_mask                       4db319a671ab6e2b099df28df89f85c4
gender                                                      Male
category                                   Masters Men 1 (40-49)
Chamonix to Delevret                                      8.6826
Chamonix to La Charme                                        NaN
Chamonix to St-Gervais                                       NaN
Delevret to St-Gervais                                   8.11842
La Charme to St-Gervais                                      NaN
St-Gervais to Contamines                                 10.7799
Contamines to La Balme                                   14.7181
La Balme to Bonhomme                                     22.7212
Bonhomme to Chapieux                                     10.8431
Chapieux to Col Seigne                                   17.9725
Col Seigne to Lac Combal                                 10.3231
Lac Combal to Mt-Favre   

In [102]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Male,Senior Men (23-39),7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Male,Masters Men 1 (40-49),8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,2015,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,2009,00156070bc3ed3fd8b6b87dabd51c2fc,Male,Senior Men (23-39),NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


# Filter out runners who only ran one race

Runner's have ran at least 2 races to be included in the case base, ie. a non personal best time and a personal best time

In [103]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [104]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
10507,2013,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
12954,2011,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.839080,NaN,NaN,2370.466667,1133.750000,13.945114
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387


# Merge

In [105]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [106]:
pb.head()

,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
10507,003cc0338235bb1862eb7b9bdd31ec9c,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
12954,006db776bb4048de7f357c632a8778ac,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
8431,00a7ade381ebb6d25e1aacb0bc7129e9,Male,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,2456.833333,1245.100000,15.121991
15237,00b39183bebaffe426e376f7efc8ae1b,Male,NaN,NaN,7.547287,NaN,NaN,9.362460,12.662551,16.833333,11.153595,13.598706,8.969388,17.229167,9.374074,10.439024,NaN,NaN,21.806122,10.963964,12.903846,19.774074,9.646667,11.486905,16.872054,NaN,NaN,15.801587,NaN,NaN,20.562893,NaN,13.461111,NaN,19.062771,NaN,NaN,14.969697,NaN,NaN,12.225225,2192.983333,899.683333,13.769848
6180,011b597a8c4121461b659f50dbd19d5a,Male,8.411765,NaN,NaN,7.717105,NaN,10.508091,12.713992,19.566667,11.323529,15.192557,23.520408,19.091667,11.370370,12.223577,29.333333,27.775,NaN,15.608108,16.535256,24.925926,11.051667,14.361905,NaN,NaN,21.212389,NaN,NaN,13.040816,25.194969,NaN,12.630556,NaN,21.954545,NaN,NaN,17.929293,NaN,NaN,12.461712,2595.100000,1325.850000,16.626208


In [107]:
npb.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387
3763,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.0,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
17546,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,29.197222,10.140777,13.462963,17.600000,11.225490,14.611650,11.180272,17.779167,10.611111,10.662602,NaN,NaN,22.261905,12.840090,13.230769,20.633333,10.486667,11.557143,17.387205,NaN,NaN,15.240741,NaN,NaN,20.226415,NaN,13.408333,NaN,19.309524,NaN,NaN,11.808081,NaN,NaN,9.322072,2255.450000,998.466667,14.964501


In [108]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [109]:
#merge = merge.groupby('name_mask').head(1)

In [110]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [111]:
first_df = len(merge)

In [112]:
merge.loc[1613]

Year                                                           2015
name_mask                          fbac5ac0b66892696a502c9f3de3ee0e
gender                                                         Male
category                                          Youth Men (20-22)
Chamonix to Delevret                                        7.48897
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                      5.99342
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                    7.80583
Contamines to La Balme                                      9.45885
La Balme to Bonhomme                                        14.3242
Bonhomme to Chapieux                                        8.26471
Chapieux to Col Seigne                                      11.6942
Col Seigne to Lac Combal                        

In [113]:
merge.groupby('category').count()

,Year,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),690,690,690,484,65,96,484,88,687,688,689,690,689,688,687,674,689,144,77,612,687,686,689,690,690,289,122,279,289,122,279,632,57,633,57,510,122,57,510,122,57,568,690,690,690,690,509,56,113,509,60,685,689,686,689,688,690,687,677,690,106,59,631,685,682,689,690,690,308,92,290,308,92,288,610,77,613,77,521,92,76,521,92,77,598,690,690,690
Masters Men 2 (50-59),290,290,290,233,14,30,233,22,290,289,290,290,289,290,288,288,288,79,39,250,289,290,290,290,290,99,42,148,99,42,149,255,34,256,34,213,42,34,213,42,34,248,290,290,290,290,167,33,67,168,51,286,289,286,288,290,290,290,283,290,32,24,265,290,283,290,290,290,164,39,87,164,39,86,268,22,268,22,228,39,22,229,39,22,251,290,290,290
Masters Men 3 (60-69),31,31,31,26,5,0,25,4,31,31,31,31,31,31,31,31,31,7,4,26,31,31,31,31,31,8,6,17,8,6,17,26,5,26,5,20,6,5,20,6,5,25,31,31,31,31,12,0,15,12,4,31,31,30,31,31,31,31,31,31,0,0,31,31,31,31,31,31,23,6,2,23,6,2,30,1,30,1,24,6,1,24,6,1,25,31,31,31
Masters Men 4 (70-79),1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1
Masters Women 1 (40-49),46,46,46,35,3,6,35,4,46,46,46,46,46,46,45,46,46,16,9,36,46,46,46,46,46,16,3,27,16,3,27,43,3,43,3,40,3,3,40,3,3,43,46,46,46,46,38,1,7,38,1,46,46,46,46,46,46,46,46,46,10,3,43,46,46,46,46,46,13,6,27,13,6,27,36,10,36,10,30,6,10,30,6,10,40,46,46,46
Masters Women 2 (50-59),14,14,14,11,1,2,11,0,14,14,14,14,14,14,14,14,14,4,1,13,14,14,14,14,14,5,2,7,5,2,7,12,2,12,2,10,2,2,10,2,2,12,14,14,14,14,12,0,1,12,1,14,14,14,14,14,14,14,14,14,0,0,14,14,14,14,14,14,7,2,5,7,2,5,13,1,13,1,11,2,1,11,2,1,12,14,14,14
Masters Women 3 (60-69),2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,1,1,1,2,2,2,2,2,0,0,2,0,0,2,2,0,2,0,2,0,0,2,0,0,2,2,2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,0,0,2,2,2,2,2,2,2,0,0,2,0,0,2,0,2,0,2,0,0,2,0,0,2,2,2,2
Senior Men (23-39),501,501,501,318,59,89,319,84,497,499,500,500,499,499,

In [115]:
#df = df[~df["AVG_Pace"] > 12 & df.category == 'Masters Men 3 (60-69)']

merge[(merge["AVG_Pace"] <12) & (merge.category == 'Masters Women 2 (50-59)')]



#df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
347,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),7.949755,NaN,NaN,5.844298,NaN,8.763754,9.91358,13.748485,7.372549,10.580906,14.908163,13.258333,7.566667,8.939024,18.055556,NaN,14.758503,8.916667,8.650641,15.911111,6.843333,8.625,NaN,NaN,11.520649,NaN,NaN,8.564626,16.606918,NaN,8.194444,NaN,17.140693,NaN,NaN,14.343434,NaN,NaN,8.497748,1753.933333,433.9,11.018994,Female,7.545343,NaN,NaN,6.116228,NaN,8.108414,9.133745,14.109091,7.745098,10.985437,6.323129,12.966667,7.181481,8.353659,NaN,NaN,16.510204,9.153153,8.86859,16.151852,7.035,8.82619,NaN,NaN,11.446903,NaN,NaN,7.55102,15.613208,NaN,7.3,NaN,15.718615,NaN,NaN,11.636364,NaN,NaN,9.177928,1667.866667,456.133333,10.148222


##### Runners with no similar runners

In [116]:
merge = merge.drop([30])
merge = merge.drop([347])

In [117]:
merge = merge[merge.name_mask != '4a40c92f6b52512ac605ea7bbcec06b6']
merge = merge[merge.name_mask != '4d5f9076cc4be3cdf23a51be2c1e7d8c']
merge = merge[merge.name_mask != '738138c60e7b32635a37f6d0757c1512']
merge = merge[merge.name_mask != '972253ab3ee99e5c6bcee7a8eba23ebf']
merge = merge[merge.name_mask != 'fbac5ac0b66892696a502c9f3de3ee0e']
merge = merge[merge.name_mask != '1e4ba9a22b3fecb9136ecfcf7f470868']
merge = merge[merge.name_mask != '32038d973773361675b8797c7a45f2bd']
merge = merge[merge.name_mask != 'c10477f93ccf5878840d9fe8b3d1291e']
merge = merge[merge.name_mask != '2ae85673eb8e4b8fc082e85b200d5064']
merge = merge[merge.name_mask != '5453919ab2a372d03ff30b1cf2494dae']
merge = merge[merge.name_mask != '3afce9f734b439fd08ff984ecbef2828']
merge = merge[merge.name_mask != 'f7f8b87a909450888d40bbb33ee52de8']
merge = merge[merge.name_mask != '3c3a924c0b32b8eda3e90af5dfd69ce7']
merge = merge[merge.name_mask != '0bf54733836ed02a86125cec3615d37c']

In [118]:
merge = merge.reset_index(drop=True)

In [119]:
merge.loc[1606]

Year                                                           2009
name_mask                          fef6b5d8143dc6816fc9a31e32e443e5
gender                                                         Male
category                                      Masters Men 1 (40-49)
Chamonix to Delevret                                            NaN
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                      7.74961
Delevret to St-Gervais                                          NaN
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                     7.8754
Contamines to La Balme                                       9.7572
La Balme to Bonhomme                                        15.5909
Bonhomme to Chapieux                                        9.04902
Chapieux to Col Seigne                                      12.0016
Col Seigne to Lac Combal                        

In [120]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [121]:
# Update pb and npb to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
npb = merge.loc[:, ~merge.columns.str.contains('pb')]

In [122]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [123]:
merge[merge.category == 'Masters Women 2 (50-59)']

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
111,2015,1320ec5e4c7a443d78f71103526fc704,Female,Masters Women 2 (50-59),10.644608,NaN,NaN,9.164474,NaN,11.508091,12.911523,20.460606,11.258170,15.555016,24.010204,24.725000,11.185185,18.715447,10.592593,28.2,NaN,14.851351,13.451923,26.985185,11.031667,15.626190,NaN,NaN,19.264012,NaN,NaN,13.520408,20.616352,NaN,10.550000,NaN,20.690476,NaN,NaN,17.156566,NaN,NaN,11.671171,2622.816667,1353.566667,16.173849,Female,8.974265,NaN,NaN,7.760965,NaN,9.928803,12.845679,19.306061,10.006536,16.221683,9.071429,19.350000,10.466667,12.544715,NaN,NaN,28.346939,12.727477,14.567308,24.359259,10.616667,12.571429,NaN,NaN,18.262537,NaN,NaN,17.734694,NaN,20.586806,NaN,23.600575,NaN,NaN,17.810811,NaN,NaN,23.142857,14.245495,2494.933333,1353.033333,15.627069
112,2013,13213e63d3265f0750495f08ab12d901,Female,Masters Women 2 (50-59),9.515931,NaN,NaN,8.736842,NaN,10.922330,11.903292,18.412121,11.101307,13.843042,10.523810,16.550000,9.881481,11.918699,NaN,NaN,23.275510,13.722973,14.701923,21.314815,11.460000,13.453571,17.031987,NaN,NaN,16.849206,NaN,NaN,24.349057,NaN,15.975000,NaN,18.474026,NaN,NaN,13.616162,NaN,NaN,8.671171,2358.233333,1123.283333,14.425177,Female,9.609069,NaN,NaN,9.385965,NaN,9.663430,11.991770,16.154545,9.983660,12.760518,8.408163,15.145833,9.540741,11.121951,NaN,NaN,21.034014,10.822072,11.253205,18.418519,12.285000,13.139286,NaN,17.222628,NaN,NaN,18.901709,NaN,21.861635,NaN,13.383333,NaN,NaN,12.624339,NaN,NaN,13.045977,NaN,NaN,2264.133333,1027.416667,13.380755
298,2016,30f977102cb8714bf2b596874ef040d2,Female,Masters Women 2 (50-59),9.572304,NaN,NaN,8.432018,NaN,11.436893,16.444444,20.839394,10.205882,18.113269,21.234694,21.037500,10.081481,13.101626,55.481481,NaN,20.336735,12.605856,12.746795,23.814815,10.713333,15.880952,NaN,NaN,22.346608,NaN,NaN,16.455782,25.698113,NaN,11.341667,NaN,21.900433,NaN,NaN,16.929293,NaN,NaN,10.472973,2670.966667,1350.933333,17.488974,Female,9.156863,NaN,NaN,7.513158,NaN,8.949838,12.718107,17.206061,7.424837,13.823625,6.894558,13.316667,10.548148,8.434959,NaN,NaN,21.061224,10.44819

In [124]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [125]:
pb.head()

,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
3,Male,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,2456.833333,1245.100000,15.121991
4,Male,NaN,NaN,7.547287,NaN,NaN,9.362460,12.662551,16.833333,11.153595,13.598706,8.969388,17.229167,9.374074,10.439024,NaN,NaN,21.806122,10.963964,12.903846,19.774074,9.646667,11.486905,16.872054,NaN,NaN,15.801587,NaN,NaN,20.562893,NaN,13.461111,NaN,19.062771,NaN,NaN,14.969697,NaN,NaN,12.225225,2192.983333,899.683333,13.769848


In [126]:
pb.to_csv('../../data/pb_times.csv')

In [127]:
npb.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387
3,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.0,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
4,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,29.197222,10.140777,13.462963,17.600000,11.225490,14.611650,11.180272,17.779167,10.611111,10.662602,NaN,NaN,22.261905,12.840090,13.230769,20.633333,10.486667,11.557143,17.387205,NaN,NaN,15.240741,NaN,NaN,20.226415,NaN,13.408333,NaN,19.309524,NaN,NaN,11.808081,NaN,NaN,9.322072,2255.450000,998.466667,14.964501


# Define Functions

In [128]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    #find all runners either 0.5 minutes/kilometre faster or slower than the query runner
   # if len(similar_times) = 1:
    #    similar_times
    return similar_times
    #else:
     #   return None

# Show sample output
find_similar_times(66)

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
3,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.000000,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
10,2016,015d1c46b9e76fa644f6a2ece59f33c0,Male,Masters Men 1 (40-49),8.468137,NaN,NaN,7.434211,NaN,10.216828,12.738683,18.975758,10.366013,16.763754,19.442177,18.466667,10.807407,12.674797,72.759259,NaN,20.782313,15.740991,13.467949,27.385185,14.000000,17.202381,NaN,NaN,21.178466,NaN,NaN,17.023810,26.893082,NaN,11.836111,NaN,24.653680,NaN,NaN,19.227273,NaN,NaN,13.317568,2725.733333,1405.700000,18.472900
27,2016,04bb6e920baaa6762c6d389ff2ace152,Male,Masters Men 1 (40-49),7.909314,NaN,NaN,6.138158,NaN,9.498382,19.415638,27.251515,7.604575,17.234628,19.727891,20.633333,10.285185,11.796748,87.240741,NaN,21.891156,14.022523,14.855769,25.640741,9.765000,13.985714,NaN,NaN,19.911504,NaN,NaN,12.122449,25.446541,NaN,7.719444,NaN,20.041126,NaN,NaN,13.217172,NaN,NaN,8.500000,2551.400000,1231.366667,18.074210
40,2015,07328403d0f0e7c8d5b3c2948cf7b6f5,Male,Masters Men 1 (40-49),8.464461,NaN,NaN,7.828947,NaN,10.799353,12.129630,17.087879,9.163399,14.440129,20.595238,17.333333,9.585185,14.235772,85.092593,22.287500,NaN,11.970721,12.647436,24.137037,11.695000,14.589286,NaN,NaN,22.323009,NaN,NaN,15.765306,25.801887,NaN,16.622222,NaN,24.761905,NaN,NaN,19.924242,NaN,NaN,16.364865,2638.700000,1369.450000,18.625853
47,2016,091369370572fe67587553de63804cf5,Male,Masters Men 1 (40-49),9.356618,NaN,NaN,8.232456,NaN,10.715210,13.676955,18.218182,11.346405,15.012945,23.986395,18.241667,10.514815,14.296748,84.537037,NaN,19.486395,13.759009,13.019231,26.140741,10.446667,14.110714,NaN,NaN,21.271386,NaN,NaN,17.874150,30.691824,NaN,13.919444,NaN,21.735931,NaN,NaN,19.898990,NaN,NaN,15.666667,2703.933333,1383.900000,19.046263
55,2015,0a407091ba797c6d47381a2c1419de32,Male,Masters Men 1 (40-49),8.148284,NaN,NaN,6.817982,NaN,9.647249,12.240741,17.087879,11.418301,14.655340,22.370748,20.150000,9.144444,11.630081,82.777778,24.079167,NaN,13.954955,15.150641,22.122222,11.416667,13.126190,NaN,NaN,20.384956,NaN,NaN,16.809524,22.710692,NaN,14.038889,NaN,21.424242,NaN,NaN,14.449495,NaN,NaN,10.421171,2500.800000,1231.550000,17.847106
66,2015,0c57fa08817368009f3f6d89823435cc,Male,Masters Men 1 (40-49),8.937500,NaN,NaN,7.546053,NaN,10.521036,14.960905,18.618182,10.160131,15.855987,24.802721,15.808333,10.048148,11.918699,75.222222,23.745833,NaN,14.869369,14.878205,22.081481,11.731667,17.026190,NaN,NaN,23.256637,NaN,NaN,13.663265,26.729560,NaN,13.205556,NaN,22.049784,NaN,NaN,20.888889,NaN,NaN,12.486486,2691.733333,1422.483333,18.440514
73,2008,0d6e8518c8d5f3e1d26d7c679f30c38b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,33.277778,10.296117,13.847737,18.627273,12.215686,15.071197,11.272109,22.266667,11.585185,13.573171,NaN,NaN,34.078231,13.439189,16.230769,25.096296,11.946

In [132]:
def cosine_similarity(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    # Return cosine similarity (1 - cosine difference)
    return 1 - (cosine(a, b))

def sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: cosine_similarity(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = False) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

sims(194)

1327    0.995193
632     0.993666
1574    0.993612
1250    0.993320
1104    0.992846
368     0.992796
1020    0.992069
1074    0.992033
1396    0.992023
1286    0.991747
dtype: float64

In [133]:
def euc_distance(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    
    return euclidean(a,b)

def euc_sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: euc_distance(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = True) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

euc_sims(194)

1327     6.871964
1104     8.560240
1574     8.816635
1396     8.842900
632      9.293761
739      9.820510
737     10.088996
1484    10.103313
673     10.248041
861     10.322933
dtype: float64

In [134]:
def weight(index):
    weighter = sims(index)
    #weight_count = len(weighter)
    #weight0 = len(str(weight_count))
   # if weight_count > 1:
   # if weighter.sum() > 3:
    if len(weighter) > 1:
        weighter = weighter / weighter.sum()
        return weighter
    
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
    #else:
     #   return np.nan
    
    #elif weight0 <= 1:
     #   return 0
    

weight(194)

1327    0.100228
632     0.100074
1574    0.100069
1250    0.100039
1104    0.099992
368     0.099986
1020    0.099913
1074    0.099910
1396    0.099909
1286    0.099881
dtype: float64

In [135]:
def euc_weight(index):
    weighter = euc_sims(index)
    if len(weighter) > 1:
        weighter = 1 -(weighter / weighter.sum())
        weighter = weighter / weighter.sum()
        return weighter
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
euc_weight(194)

1327    0.102898
1104    0.100880
1574    0.100574
1396    0.100543
632     0.100004
739     0.099374
737     0.099053
1484    0.099036
673     0.098863
861     0.098774
dtype: float64

In [137]:
def find_nearest_neighbours(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    else: 
        nearest_neighbours['weights'] = weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        
        return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(194)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,weights,weighted_pace
1327,2017,cf40c71cb6d621ff47e587e5edfbfc13,Male,Masters Men 1 (40-49),10.113971,NaN,NaN,8.949561,NaN,12.328479,14.623457,20.951515,12.954248,17.454693,10.323129,22.787500,12.448148,13.800813,NaN,NaN,26.772109,15.254505,12.903846,31.496296,13.253333,13.648810,NaN,NaN,22.368732,NaN,NaN,21.445578,NaN,23.416667,NaN,18.002874,NaN,NaN,19.536036,NaN,NaN,20.726190,14.747748,2744.100000,1602.200000,17.096177,Male,9.579657,NaN,NaN,9.076754,NaN,11.334951,14.903292,19.678788,12.751634,16.493528,10.489796,22.387500,12.000000,13.239837,NaN,NaN,29.993197,14.414414,16.496795,26.085185,13.076667,16.310714,NaN,NaN,20.179941,NaN,NaN,18.391156,27.506289,NaN,13.413889,NaN,21.989177,NaN,NaN,17.257576,NaN,NaN,11.146396,2698.200000,1486.466667,16.591547,0.100228,1.662935
632,2011,64560c9de4b1a2a57b536572145705de,Male,Masters Men 1 (40-49),9.447304,NaN,NaN,8.250000,NaN,11.457929,16.146091,18.430303,9.434641,17.660194,9.476190,24.004167,10.400000,12.817073,NaN,NaN,29.112245,13.002252,13.772436,28.114815,17.401667,15.303571,NaN,19.255474,NaN,NaN,21.117521,NaN,25.132075,NaN,10.263889,NaN,NaN,13.044974,NaN,NaN,15.833333,NaN,NaN,2681.666667,1444.950000,16.038180,Male,7.629902,NaN,NaN,6.245614,NaN,8.454693,12.658436,15.730303,7.630719,14.368932,7.884354,19.470833,8.570370,13.203252,NaN,NaN,24.275510,10.864865,13.839744,29.659259,9.140000,12.283333,NaN,NaN,21.233038,NaN,NaN,17.527211,NaN,23.920139,NaN,18.428161,NaN,NaN,25.63964,NaN,NaN,23.190476,12.695946,2390.316667,1248.416667,15.189364,0.100074,1.520062
1574,2017,fa1d10da48a57c2693cd147032e6bbb3,Male,Masters Men 1 (40-49),9.426471,NaN,NaN,8.026316,NaN,10.430421,14.927984,20.433333,11.294118,16.561489,9.666667,21.275000,10.011111,12.922764,NaN,NaN,26.823129,13.454955,15.032051,27.588889,11.580000,13.842857,NaN,NaN,23.718289,NaN,NaN,18.367347,NaN,34.270833,NaN,18.574713,NaN,NaN,24.738739,NaN,NaN,21.171429,15.067568,2713.016667,1571.116667,17.050270,Male,8.774510,NaN,NaN,7.195175,NaN,11.831715,11.471193,18.378788,11.130719,15.326861,10.717687,19.504167,11.57407

In [138]:
def euc_find_nearest_neighbours(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1: 
        nearest_neighbours['weights'] = euc_weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    else: 
        nearest_neighbours['weights'] = euc_weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb'] 
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
# Show sample output    
euc_find_nearest_neighbours(194)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,weights,weighted_pace
1327,2017,cf40c71cb6d621ff47e587e5edfbfc13,Male,Masters Men 1 (40-49),10.113971,NaN,NaN,8.949561,NaN,12.328479,14.623457,20.951515,12.954248,17.454693,10.323129,22.787500,12.448148,13.800813,NaN,NaN,26.772109,15.254505,12.903846,31.496296,13.253333,13.648810,NaN,NaN,22.368732,NaN,NaN,21.445578,NaN,23.416667,NaN,18.002874,NaN,NaN,19.536036,NaN,NaN,20.726190,14.747748,2744.100000,1602.200000,17.096177,Male,9.579657,NaN,NaN,9.076754,NaN,11.334951,14.903292,19.678788,12.751634,16.493528,10.489796,22.387500,12.000000,13.239837,NaN,NaN,29.993197,14.414414,16.496795,26.085185,13.076667,16.310714,NaN,NaN,20.179941,NaN,NaN,18.391156,27.506289,NaN,13.413889,NaN,21.989177,NaN,NaN,17.257576,NaN,NaN,11.146396,2698.200000,1486.466667,16.591547,0.102898,1.707240
1104,2017,ad2738ff824ba742eb71ee52c8cb41dc,Male,Masters Men 1 (40-49),10.273284,NaN,NaN,9.581140,NaN,11.627832,16.419753,23.421212,12.604575,19.058252,10.037415,23.450000,12.588889,11.402439,NaN,NaN,29.394558,16.279279,11.291667,30.037037,11.736667,13.823810,NaN,NaN,22.581121,NaN,NaN,17.040816,NaN,24.590278,NaN,17.790230,NaN,NaN,17.018018,NaN,NaN,20.719048,15.301802,2751.483333,1609.583333,17.002880,Male,9.998775,NaN,NaN,8.561404,NaN,11.966019,14.152263,21.066667,12.650327,18.736246,10.806122,20.416667,11.977778,12.719512,NaN,NaN,30.448980,17.047297,13.246795,31.562963,12.458333,16.304762,20.186869,NaN,NaN,17.103175,NaN,NaN,24.408805,NaN,12.119444,NaN,23.099567,NaN,NaN,13.818182,NaN,NaN,13.004505,2721.466667,1486.516667,16.577561,0.100880,1.672352
1574,2017,fa1d10da48a57c2693cd147032e6bbb3,Male,Masters Men 1 (40-49),9.426471,NaN,NaN,8.026316,NaN,10.430421,14.927984,20.433333,11.294118,16.561489,9.666667,21.275000,10.011111,12.922764,NaN,NaN,26.823129,13.454955,15.032051,27.588889,11.580000,13.842857,NaN,NaN,23.718289,NaN,NaN,18.367347,NaN,34.270833,NaN,18.574713,NaN,NaN,24.738739,NaN,NaN,21.171429,15.067568,2713.016667,1571.116667,17.050270,Male,8.774510,NaN,NaN,7.195175,NaN,11.831715,11.471193,18.378788,11.130719,15.326861,10.717687,19

### Pacing Plan Recommendation

In [139]:
def find_nearest_neighbours_pace(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    #nearest_neighbours = nearest_neighbours.filter(like='to')
    #nearest_neighbours['weights'] = weight(index)
    #nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = nearest_neighbours.loc[:, 'Chamonix to Delevret_pb':'Flegere to Chamonix_pb'].mul(nearest_neighbours['weighted_pace'], axis=0)
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours_pace(194)

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
1327,2017,cf40c71cb6d621ff47e587e5edfbfc13,Male,Masters Men 1 (40-49),10.113971,NaN,NaN,8.949561,NaN,12.328479,14.623457,20.951515,12.954248,17.454693,10.323129,22.787500,12.448148,13.800813,NaN,NaN,26.772109,15.254505,12.903846,31.496296,13.253333,13.648810,NaN,NaN,22.368732,NaN,NaN,21.445578,NaN,23.416667,NaN,18.002874,NaN,NaN,19.536036,NaN,NaN,20.726190,14.747748,2744.100000,1602.200000,17.096177,Male,9.579657,NaN,NaN,9.076754,NaN,11.334951,14.903292,19.678788,12.751634,16.493528,10.489796,22.387500,12.000000,13.239837,NaN,NaN,29.993197,14.414414,16.496795,26.085185,13.076667,16.310714,NaN,NaN,20.179941,NaN,NaN,18.391156,27.506289,NaN,13.413889,NaN,21.989177,NaN,NaN,17.257576,NaN,NaN,11.146396,2698.200000,1486.466667,16.591547
632,2011,64560c9de4b1a2a57b536572145705de,Male,Masters Men 1 (40-49),9.447304,NaN,NaN,8.250000,NaN,11.457929,16.146091,18.430303,9.434641,17.660194,9.476190,24.004167,10.400000,12.817073,NaN,NaN,29.112245,13.002252,13.772436,28.114815,17.401667,15.303571,NaN,19.255474,NaN,NaN,21.117521,NaN,25.132075,NaN,10.263889,NaN,NaN,13.044974,NaN,NaN,15.833333,NaN,NaN,2681.666667,1444.950000,16.038180,Male,7.629902,NaN,NaN,6.245614,NaN,8.454693,12.658436,15.730303,7.630719,14.368932,7.884354,19.470833,8.570370,13.203252,NaN,NaN,24.275510,10.864865,13.839744,29.659259,9.140000,12.283333,NaN,NaN,21.233038,NaN,NaN,17.527211,NaN,23.920139,NaN,18.428161,NaN,NaN,25.63964,NaN,NaN,23.190476,12.695946,2390.316667,1248.416667,15.189364
1574,2017,fa1d10da48a57c2693cd147032e6bbb3,Male,Masters Men 1 (40-49),9.426471,NaN,NaN,8.026316,NaN,10.430421,14.927984,20.433333,11.294118,16.561489,9.666667,21.275000,10.011111,12.922764,NaN,NaN,26.823129,13.454955,15.032051,27.588889,11.580000,13.842857,NaN,NaN,23.718289,NaN,NaN,18.367347,NaN,34.270833,NaN,18.574713,NaN,NaN,24.738739,NaN,NaN,21.171429,15.067568,2713.016667,1571.116667,17.050270,Male,8.774510,NaN,NaN,7.195175,NaN,11.831715,11.471193,18.378788,11.130719,15.326861,10.717687,19.504167,11.574074,11.463415,NaN,NaN,23.721088,13.490991,15.394231,25.55555

In [140]:
def euc_find_nearest_neighbours_pace(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = euc_weight(index)
        #nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        nearest_neighbours = nearest_neighbours.loc[:, 'Chamonix to Delevret_pb':'Flegere to Chamonix_pb'].mul(nearest_neighbours['weights'], axis=0)
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    
    
# Show sample output    
euc_find_nearest_neighbours_pace(194)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
1327,0.985729,NaN,NaN,0.933981,NaN,1.166346,1.533521,2.024911,1.312120,1.697154,1.079381,2.303633,1.234778,1.362355,NaN,NaN,3.086245,1.483217,1.697490,2.684118,1.345565,1.678343,NaN,NaN,2.076479,NaN,NaN,1.892416,2.830347,NaN,1.380265,NaN,2.262646,NaN,NaN,1.775773,NaN,NaN,1.146944
1104,1.008681,NaN,NaN,0.863678,NaN,1.207137,1.427687,2.125215,1.276171,1.890121,1.090126,2.059642,1.208324,1.283150,NaN,NaN,3.071707,1.719739,1.336343,3.184086,1.256802,1.644832,2.036460,NaN,NaN,1.725376,NaN,NaN,2.462371,NaN,1.222615,NaN,2.330295,NaN,NaN,1.393984,NaN,NaN,1.311900
1574,0.882488,NaN,NaN,0.723648,NaN,1.189963,1.153704,1.848429,1.119461,1.541484,1.077921,1.961612,1.164051,1.152922,NaN,NaN,2.385725,1.356843,1.548260,2.570225,1.193478,1.452696,2.268164,NaN,NaN,1.807938,NaN,NaN,2.778753,NaN,1.677630,NaN,3.164163,NaN,NaN,1.567024,NaN,NaN,1.404412
1396,1.019842,NaN,NaN,0.842706,NaN,1.260850,1.322363,2.256116,1.310011,1.710201,1.138457,2.208167,1.105224,1.207329,NaN,NaN,2.767316,1.407597,1.485260,2.808118,1.212209,1.746090,1.761527,NaN,NaN,1.778115,NaN,NaN,2.693784,NaN,1.205953,NaN,2.236094,NaN,NaN,1.371036,NaN,NaN,1.163487
632,0.763019,NaN,NaN,0.624585,NaN,0.845501,1.265892,1.573090,0.763101,1.436948,0.788465,1.947157,0.857069,1.320375,NaN,NaN,2.427643,1.086528,1.384027,2.966038,0.914035,1.228380,NaN,NaN,2.123384,NaN,NaN,1.752787,NaN,2.392104,NaN,1.842886,NaN,NaN,2.564061,NaN,NaN,2.319135,1.269643
739,0.847482,NaN,NaN,0.765574,NaN,1.097298,1.568109,1.970621,1.197362,1.587898,0.957576,2.056219,1.050791,1.410630,NaN,NaN,3.096150,1.358786,1.417039,2.639306,1.120279,1.597678,NaN,NaN,2.016066,NaN,NaN,1.594720,2.693105,NaN,1.329959,NaN,2.367774,NaN,NaN,1.796767,NaN,NaN,1.269036
737,0.844746,NaN,NaN,0.763102,NaN,1.093754,1.563046,1.964258,1.193496,1.582770,0.954484,2.049579,1.047398,1.406075,NaN,NaN,3.086153,1.354399,1.412463,2.630784,1.116662,1.592519,NaN,NaN,2.009556,NaN,NaN,1.589571,2.684409,NaN,1.325664,NaN,2.360129,NaN,NaN,1.790965,NaN,NaN,1.264938
1484,NaN,NaN,0.907986,NaN,NaN,1.023054,1.233266,2.006835,1.280351,1.610221,1.031796,1.860231,0.927640,1.104295,NaN,NaN,2.814449,1.420189,1.471577,2.184667,1.195038,1.636810,2.015238,NaN,NaN,1.924657,NaN,NaN,2.822533,NaN,1.459684,NaN,2.037532,NaN,NaN,1.705124,NaN,NaN,1.148285
673,0.862752,NaN,NaN,0.791123,NaN,0.856015,1.103363,1.728909,0.899785,1.597330,0.890442,1.982621,1.039895,1.135320,NaN,NaN,3.500568,1.346456,1.577060,2.237606,1.615756,1.728342,NaN,1.957421,NaN,NaN,1.941565,NaN,2.271058,NaN,1.376397,NaN,NaN,1.343547,NaN,NaN,1.420828,NaN,NaN
861,0.850592,NaN,NaN,0.638347,NaN,0.987258,1.121062,1.893464,1.184317,1.431101,0.886948,1.873409,1.080293,1.365568,NaN,NaN,2.378298,1.318541,1.450582,2.445200,1.057373,1.331682,1.517691,NaN,NaN,1.812159,NaN,NaN,2.313108,NaN,1.338659,NaN,2.299377,NaN,NaN,1.351904,NaN,NaN,1.091851


In [141]:
def weighted_row(neighbours):
    #row_pace = find_nearest_neighbours_pace(index)
    row_pace = neighbours.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    #pred_pace = row_pace
    return row_pace

neighbours = find_nearest_neighbours_pace(194)
weighted_row(neighbours)

,Year,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014.6,9.746814,NaN,NaN,9.036184,NaN,10.973139,14.309877,19.962121,11.37549,16.870388,10.061905,21.231667,11.148148,12.948374,NaN,NaN,27.184014,13.734009,13.912821,27.504074,13.4365,14.470357,NaN,18.166058,23.292035,NaN,18.793803,18.353741,23.540094,27.771412,11.383333,19.079502,NaN,12.91336,19.967718,NaN,15.032567,21.888889,14.12988,2654.643333,1474.816667,16.286844,8.947032,5.576812,NaN,7.79191,10.536111,10.706472,12.910905,18.680606,11.260784,15.653883,11.47483,19.851667,10.833704,12.297154,36.0,20.891667,26.707483,13.46464,13.621795,24.632593,11.011167,14.440595,20.23064,NaN,19.026549,17.642328,NaN,15.189116,27.383298,23.920139,14.008333,18.428161,23.326359,NaN,25.63964,15.714927,NaN,23.190476,11.270495,2522.306667,1297.993333,15.604234


In [142]:
def pace_rec(index):
    #print(index)
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

pace_rec(0)

,Year,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2012.8,8.691789,4.914493,9.158915,7.341831,8.447222,9.729612,12.251029,17.236667,9.670588,14.398058,9.007823,18.76375,9.616296,11.20122,NaN,NaN,25.891156,11.175901,12.610256,22.06037,10.777667,13.304643,16.511785,20.819951,18.671583,13.861552,20.17094,13.710884,22.415793,17.086806,13.357099,16.485632,23.068723,15.526455,19.490991,16.32702,15.300766,18.140476,10.264264,2323.768333,1101.516667,14.223918,7.836737,NaN,NaN,6.335283,28.202778,8.832524,11.070165,15.821212,8.595752,13.205016,11.221429,17.768333,8.515226,10.693496,37.425926,20.3,22.432398,10.576076,12.314744,20.877407,8.790833,10.931786,16.653199,NaN,15.695346,13.740741,NaN,10.718065,20.789308,18.467593,11.267063,14.055556,19.682746,NaN,17.593093,15.649351,NaN,17.396032,9.502252,2067.878333,850.236667,13.330479


In [143]:
def weighted_rows(index):
    #print(index)
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

weighted_rows(194)

,Year,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
194,2014.6,9.746814,NaN,NaN,9.036184,NaN,10.973139,14.309877,19.962121,11.37549,16.870388,10.061905,21.231667,11.148148,12.948374,NaN,NaN,27.184014,13.734009,13.912821,27.504074,13.4365,14.470357,NaN,18.166058,23.292035,NaN,18.793803,18.353741,23.540094,27.771412,11.383333,19.079502,NaN,12.91336,19.967718,NaN,15.032567,21.888889,14.12988,2654.643333,1474.816667,16.286844,8.947032,5.576812,NaN,7.79191,10.536111,10.706472,12.910905,18.680606,11.260784,15.653883,11.47483,19.851667,10.833704,12.297154,36.0,20.891667,26.707483,13.46464,13.621795,24.632593,11.011167,14.440595,20.23064,NaN,19.026549,17.642328,NaN,15.189116,27.383298,23.920139,14.008333,18.428161,23.326359,NaN,25.63964,15.714927,NaN,23.190476,11.270495,2522.306667,1297.993333,15.604234


In [145]:
weighted_rows(4)

,Year,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
4,2010.8,9.305147,8.45942,7.739922,8.669956,30.567593,10.193528,13.145267,18.353333,11.215686,15.354693,9.964626,17.893333,10.364074,11.469512,NaN,NaN,24.955442,13.107432,13.883654,22.291481,11.438833,12.649048,17.908129,17.212895,18.269912,15.535525,19.638889,15.147959,22.28066,20.859375,13.501389,19.156609,20.827458,12.140212,21.425676,13.518038,16.172414,20.194048,10.587087,2378.333333,1141.333333,15.028079,8.558211,4.165217,8.368475,7.451389,8.075,9.46068,11.325926,17.049697,9.704248,13.576699,8.768027,17.410417,9.752593,11.40935,NaN,NaN,24.52585,11.846847,12.151282,21.448519,10.8375,12.920357,17.216498,16.950122,16.79351,15.209524,17.297009,14.352041,22.722013,NaN,13.229722,NaN,22.717069,11.358025,NaN,17.119048,12.376756,NaN,12.502896,2280.203333,1029.658333,14.010092


####  Cosine Weighted Rows

In [146]:
ap = pb['AVG_Pace_pb']

In [147]:
ap = pd.DataFrame(ap)

In [148]:
ap = ap.rename(index=str, columns={"AVG_Pace_pb": "AVG_Pace_pb1", })

In [149]:
pb1 = pb.filter(like='to')

In [150]:
pb1.shape

(1611, 39)

In [151]:
weighted_pb = pd.DataFrame()

In [152]:
weighted_pb

""


In [153]:
pb1.shape[0]

1611

In [154]:
for i in range(pb1.shape[0]):
    weighted_pb = weighted_pb.append(pace_rec(i), ignore_index=True)

In [155]:
weighted_pb

,Year,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2012.800000,8.691789,4.914493,9.158915,7.341831,8.447222,9.729612,12.251029,17.236667,9.670588,14.398058,9.007823,18.763750,9.616296,11.201220,NaN,NaN,25.891156,11.175901,12.610256,22.060370,10.777667,13.304643,16.511785,20.819951,18.671583,13.861552,20.170940,13.710884,22.415793,17.086806,13.357099,16.485632,23.068723,15.526455,19.490991,16.327020,15.300766,18.140476,10.264264,2323.768333,1101.516667,14.223918,7.836737,NaN,NaN,6.335283,28.202778,8.832524,11.070165,15.821212,8.595752,13.205016,11.221429,17.768333,8.515226,10.693496,37.425926,20.300000,22.432398,10.576076,12.314744,20.877407,8.790833,10.931786,16.653199,NaN,15.695346,13.740741,NaN,10.718065,20.789308,18.467593,11.267063,14.055556,19.682746,NaN,17.593093,15.649351,NaN,17.396032,9.502252,2067.878333,850.236667,13.330479
1,2010.600000,9.334209,5.976087,8.971705,8.919173,11.922222,10.292395,13.362551,18.252121,10.433333,15.269094,9.521088,19.537917,10.541111,13.425610,NaN,NaN,27.596599,13.147973,15.244231,25.055926,14.028500,16.300000,21.609989,19.285483,18.572271,17.339506,17.272436,15.836735,22.048428,NaN,12.487222,NaN,20.978896,12.954586,NaN,16.127525,13.503193,NaN,12.064189,2539.125000,1291.563333,15.317765,8.689706,NaN,NaN,7.093640,NaN,10.362298,11.794239,17.091515,9.876797,14.109061,11.346599,18.755000,9.511852,11.906504,32.453704,NaN,24.389456,12.237838,12.509615,22.098148,10.468167,13.007143,15.813973,NaN,19.917035,14.761905,NaN,14.133503,23.532704,NaN,12.920278,NaN,22.824675,NaN,NaN,15.912626,NaN,NaN,11.209459,2334.395000,1087.071667,14.455599
2,2011.700000,9.098775,5.976087,8.048256,8.463596,11.922222,9.835761,12.683333,17.773333,10.952941,14.598706,9.633673,18.392083,9.890000,12.179675,NaN,NaN,26.809524,13.402027,14.174038,24.559259,11.771500,14.891667,22.232323,18.916058,20.218289,19.632804,19.886752,19.151361,24.957547,24.579861,15.720486,18.550287,26.322202,13.587302,52.322072,17.893218,15.919540,19.422619,15.251001,2590.085000,1352.668333,16.129044,8.762255,5.963043,NaN,7.414748,19.631944,10.182686,11.601029,16.756061,9.528431,13.983981,12.396599,17.955833,9

In [156]:
weighted_pb = comb.join(weighted_pb)

In [157]:
weighted_pb

,category,gender,Year,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,Senior Men (23-39),Male,2012.800000,8.691789,4.914493,9.158915,7.341831,8.447222,9.729612,12.251029,17.236667,9.670588,14.398058,9.007823,18.763750,9.616296,11.201220,NaN,NaN,25.891156,11.175901,12.610256,22.060370,10.777667,13.304643,16.511785,20.819951,18.671583,13.861552,20.170940,13.710884,22.415793,17.086806,13.357099,16.485632,23.068723,15.526455,19.490991,16.327020,15.300766,18.140476,10.264264,2323.768333,1101.516667,14.223918,7.836737,NaN,NaN,6.335283,28.202778,8.832524,11.070165,15.821212,8.595752,13.205016,11.221429,17.768333,8.515226,10.693496,37.425926,20.300000,22.432398,10.576076,12.314744,20.877407,8.790833,10.931786,16.653199,NaN,15.695346,13.740741,NaN,10.718065,20.789308,18.467593,11.267063,14.055556,19.682746,NaN,17.593093,15.649351,NaN,17.396032,9.502252,2067.878333,850.236667,13.330479
1,Senior Men (23-39),Male,2010.600000,9.334209,5.976087,8.971705,8.919173,11.922222,10.292395,13.362551,18.252121,10.433333,15.269094,9.521088,19.537917,10.541111,13.425610,NaN,NaN,27.596599,13.147973,15.244231,25.055926,14.028500,16.300000,21.609989,19.285483,18.572271,17.339506,17.272436,15.836735,22.048428,NaN,12.487222,NaN,20.978896,12.954586,NaN,16.127525,13.503193,NaN,12.064189,2539.125000,1291.563333,15.317765,8.689706,NaN,NaN,7.093640,NaN,10.362298,11.794239,17.091515,9.876797,14.109061,11.346599,18.755000,9.511852,11.906504,32.453704,NaN,24.389456,12.237838,12.509615,22.098148,10.468167,13.007143,15.813973,NaN,19.917035,14.761905,NaN,14.133503,23.532704,NaN,12.920278,NaN,22.824675,NaN,NaN,15.912626,NaN,NaN,11.209459,2334.395000,1087.071667,14.455599
2,Senior Men (23-39),Male,2011.700000,9.098775,5.976087,8.048256,8.463596,11.922222,9.835761,12.683333,17.773333,10.952941,14.598706,9.633673,18.392083,9.890000,12.179675,NaN,NaN,26.809524,13.402027,14.174038,24.559259,11.771500,14.891667,22.232323,18.916058,20.218289,19.632804,19.886752,19.151361,24.957547,24.579861,15.720486,18.550287,26.322202,13.587302,52.322072,17.893218,15.919540,19.422619,15.251001,2590.085000,1352.668333,16.129044,8.762255,5.963043,NaN,7

In [158]:
# Read in Split Timings
weighted_pb.to_csv('../../data/weighted_pb.csv', encoding = "ISO-8859-1")

In [159]:
# Read in Split Timings
w_pb = pd.read_csv('../../data/weighted_pb.csv', encoding = "ISO-8859-1")

In [160]:
w_pb.iloc[[8]]

,Unnamed: 0,category,gender,Year,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
8,8,Senior Men (23-39),Male,2010.8,9.337469,8.773188,9.208527,8.708882,NaN,10.330097,13.347737,17.896667,9.824837,15.102104,9.714286,19.158333,10.326667,12.985366,NaN,NaN,30.522449,13.156982,15.176923,24.581111,14.444833,16.972976,21.958754,19.899722,22.750737,18.869048,16.96337,14.993197,22.956289,NaN,11.604722,NaN,20.209957,12.404006,NaN,17.324916,12.917077,NaN,12.307057,2553.945,1312.041667,15.354854,8.683824,5.332609,8.513695,6.843202,9.122222,10.057929,12.803292,17.311515,9.630719,14.545307,11.343197,18.47125,9.432963,10.848374,45.416667,NaN,26.387075,12.882883,13.644231,22.196667,10.283167,13.352381,18.612554,NaN,19.372665,15.394936,NaN,13.93424,23.867296,NaN,13.104167,NaN,22.111688,NaN,NaN,15.558586,NaN,NaN,10.452703,2373.073333,1103.72,14.82632


####  means

In [167]:
def get_mean_pb(neighbours):
    mean = neighbours.AVG_Pace_pb.mean() 
    return mean
    
# Show sample output    
neighbours = find_nearest_neighbours(10)
get_mean_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.246859541591721

In [168]:
def get_weight_mean(neighbours):
    w_mean = neighbours.weighted_pace.sum()
    return w_mean

neighbours = find_nearest_neighbours(10)
get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.246495219638165

In [169]:
def euc_get_weight_mean(neighbours):
    ew_mean = neighbours.weighted_pace.sum()
    return ew_mean

neighbours = euc_find_nearest_neighbours(10)
euc_get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.206456160476971

In [171]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_best_npb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


16.223586238176683

##  Best pace row

In [172]:
def best_pace_rec(index):
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.sort_values(by='AVG_Pace_pb', ascending=True)
    row_pace = row_pace.head(1)
   # row_pace = pd.DataFrame(row_pace)
   # row_pace = row_pace.T
    row_pace = row_pace.filter(like='to')
    row_pace.index = query.index

    return row_pace


best_pace_rec(5)

,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
5,7.546569,NaN,NaN,6.175439,NaN,12.5,10.691358,17.966667,8.895425,13.865696,8.962585,17.604167,10.774074,12.768293,NaN,NaN,25.809524,12.211712,13.974359,22.711111,11.751667,14.441667,17.181818,NaN,NaN,18.505291,NaN,NaN,30.465409,NaN,14.897222,NaN,25.32684,NaN,NaN,14.757576,NaN,NaN,13.572072,7.112745,NaN,NaN,5.438596,NaN,7.991909,12.532922,17.493939,8.19281,13.949838,8.193878,19.404167,12.262963,16.052846,NaN,NaN,28.387755,11.144144,10.416667,22.488889,9.268333,11.782143,NaN,NaN,16.597345,NaN,NaN,10.418367,23.059748,NaN,9.977778,NaN,20.448052,NaN,NaN,19.388889,NaN,NaN,10.265766


In [173]:
pb_top = pd.DataFrame()

In [174]:
pb_top

""


In [175]:
for i in range(pb1.shape[0]):
    pb_top = pb_top.append(best_pace_rec(i), ignore_index=True)

In [176]:
pb_top

,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,NaN,NaN,9.158915,NaN,NaN,10.208738,12.047325,18.227273,11.359477,14.207120,8.238095,17.487500,8.911111,12.333333,NaN,NaN,23.272109,10.795045,11.310897,19.744444,11.431667,11.636905,16.858586,NaN,NaN,13.838624,NaN,NaN,20.877358,NaN,12.711111,NaN,22.694805,NaN,NaN,12.479798,NaN,NaN,8.990991,7.279412,NaN,NaN,5.923246,NaN,7.634304,9.102881,14.621212,8.967320,12.095469,7.197279,16.962500,7.866667,9.772358,NaN,NaN,18.472789,9.218468,9.237179,18.862963,7.588333,9.205952,NaN,NaN,12.384956,NaN,NaN,8.275510,16.427673,NaN,8.019444,NaN,16.435065,NaN,NaN,13.287879,NaN,NaN,7.677928
1,9.731618,NaN,NaN,9.361842,NaN,11.273463,15.283951,16.545455,9.238562,14.444984,9.931973,20.262500,9.429630,14.000000,NaN,NaN,29.510204,10.777027,16.185897,22.733333,15.156667,18.977381,NaN,19.045012,NaN,NaN,17.429487,NaN,23.003145,NaN,11.383333,NaN,NaN,12.642857,NaN,NaN,12.852490,NaN,NaN,8.417892,NaN,NaN,6.309211,NaN,9.674757,9.057613,15.342424,9.045752,12.006472,7.489796,16.054167,8.718519,10.813008,NaN,NaN,23.221088,10.310811,10.564103,16.733333,8.940000,10.070238,13.207071,NaN,NaN,11.817460,NaN,NaN,18.462264,NaN,8.583333,NaN,17.110390,NaN,NaN,11.202020,NaN,NaN,9.560811
2,9.446078,NaN,NaN,9.598684,NaN,10.668285,13.032922,18.666667,12.267974,14.385113,10.574830,19.554167,10.596296,13.601626,NaN,NaN,27.816327,12.986486,12.875000,25.533333,12.008333,13.936905,NaN,NaN,20.045723,NaN,NaN,15.017007,24.808176,NaN,15.894444,NaN,24.872294,NaN,NaN,18.505051,NaN,NaN,12.918919,8.191176,NaN,NaN,7.311404,NaN,8.351133,10.779835,15.254545,8.477124,12.911003,7.744898,15.508333,8.562963,12.020325,NaN,NaN,25.275510,11.774775,11.464744,21.774074,11.990000,14.207143,NaN,19.768856,NaN,NaN,21.151709,NaN,23.421384,NaN,11.538889,NaN,NaN,13.142857,NaN,NaN,12.126437,NaN,NaN
3,9.280637,NaN,NaN,7.666667,NaN,10.334951,12.697531,16.703030,10.078431,14.818770,17.979592,18.733333,9.670370,13.329268,46.592593,NaN,19.904762,13.101351,12.205128,22.159259,11.156667,13.450000,NaN,NaN,23.423304,NaN,NaN,16.727891,25.540881,NaN,15.950000,NaN,23.075758,NaN,NaN,18.898990,NaN,NaN,12.472973,7.908088,NaN,NaN,6.151316,NaN,9.098706,10.203704,15.051515,8.078431,13.184466,7.9

In [177]:
pb_top = comb.join(pb_top)

In [178]:
pb_top

,category,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,Senior Men (23-39),Male,NaN,NaN,9.158915,NaN,NaN,10.208738,12.047325,18.227273,11.359477,14.207120,8.238095,17.487500,8.911111,12.333333,NaN,NaN,23.272109,10.795045,11.310897,19.744444,11.431667,11.636905,16.858586,NaN,NaN,13.838624,NaN,NaN,20.877358,NaN,12.711111,NaN,22.694805,NaN,NaN,12.479798,NaN,NaN,8.990991,7.279412,NaN,NaN,5.923246,NaN,7.634304,9.102881,14.621212,8.967320,12.095469,7.197279,16.962500,7.866667,9.772358,NaN,NaN,18.472789,9.218468,9.237179,18.862963,7.588333,9.205952,NaN,NaN,12.384956,NaN,NaN,8.275510,16.427673,NaN,8.019444,NaN,16.435065,NaN,NaN,13.287879,NaN,NaN,7.677928
1,Senior Men (23-39),Male,9.731618,NaN,NaN,9.361842,NaN,11.273463,15.283951,16.545455,9.238562,14.444984,9.931973,20.262500,9.429630,14.000000,NaN,NaN,29.510204,10.777027,16.185897,22.733333,15.156667,18.977381,NaN,19.045012,NaN,NaN,17.429487,NaN,23.003145,NaN,11.383333,NaN,NaN,12.642857,NaN,NaN,12.852490,NaN,NaN,8.417892,NaN,NaN,6.309211,NaN,9.674757,9.057613,15.342424,9.045752,12.006472,7.489796,16.054167,8.718519,10.813008,NaN,NaN,23.221088,10.310811,10.564103,16.733333,8.940000,10.070238,13.207071,NaN,NaN,11.817460,NaN,NaN,18.462264,NaN,8.583333,NaN,17.110390,NaN,NaN,11.202020,NaN,NaN,9.560811
2,Senior Men (23-39),Male,9.446078,NaN,NaN,9.598684,NaN,10.668285,13.032922,18.666667,12.267974,14.385113,10.574830,19.554167,10.596296,13.601626,NaN,NaN,27.816327,12.986486,12.875000,25.533333,12.008333,13.936905,NaN,NaN,20.045723,NaN,NaN,15.017007,24.808176,NaN,15.894444,NaN,24.872294,NaN,NaN,18.505051,NaN,NaN,12.918919,8.191176,NaN,NaN,7.311404,NaN,8.351133,10.779835,15.254545,8.477124,12.911003,7.744898,15.508333,8.562963,12.020325,NaN,NaN,25.275510,11.774775,11.464744,21.774074,11.990000,14.207143,NaN,19.768856,NaN,NaN,21.151709,NaN,23.421384,NaN,11.538889,NaN,NaN,13.142857,NaN,NaN,12.126437,NaN,NaN
3,Masters Men 1 (40-49),Male,9.280637,NaN,NaN,7.666667,NaN,10.334951,12.697531,16.703030,10.078431,14.818770,17.979592,18.733333,9.670370,13.329268,46.592593,NaN,19.904762,13.101351,12.205128,22.159259,11.156667,13.450000,NaN,NaN,23.423304,NaN,NaN,16.727891,25.540881,NaN,15.950000,NaN,23.075758,Na

In [179]:
# Read in Split Timings
pb_top.to_csv('../../data/pb_top.csv', encoding = "ISO-8859-1")

In [180]:
# Read in Split Timings
pb_top = pd.read_csv('../../data/pb_top.csv', encoding = "ISO-8859-1")

In [181]:
pb_top.iloc[[8]]

,Unnamed: 0,category,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
8,8,Senior Men (23-39),Male,8.571078,NaN,NaN,7.776316,NaN,8.699029,12.111111,17.251515,9.287582,13.995146,7.92517,19.620833,8.977778,11.743902,NaN,NaN,29.741497,12.871622,13.13141,22.859259,15.068333,15.546429,NaN,19.812652,NaN,NaN,17.758547,NaN,21.481132,NaN,10.369444,NaN,NaN,11.07672,NaN,NaN,11.850575,NaN,NaN,8.493873,NaN,NaN,6.342105,NaN,10.017799,10.621399,15.084848,8.51634,12.847896,7.812925,16.491667,8.82963,11.239837,NaN,NaN,21.190476,10.695946,11.512821,20.737037,9.073333,12.221429,13.811448,NaN,NaN,12.404762,NaN,NaN,21.933962,NaN,10.672222,NaN,20.190476,NaN,NaN,14.787879,NaN,NaN,11.461712


## Get Best

In [182]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(5)
get_best_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


13.844603742224985

In [183]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


11.680842484832413

# Calculate Predictions

In [184]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
   # row['weighted_mean_pb_ap'] = weighted_pace(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    row['weighted_mean'] = get_weight_mean(neighbours)
    neighbours1 = euc_find_nearest_neighbours(index)
    row['euc_weighted_mean'] = euc_get_weight_mean(neighbours1)
   # row['ranked_mean'] = get_rank_mean(neighbours)
    return row

In [185]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data,  'weighted_mean':filler_data,   'euc_weighted_mean':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace', 'weighted_mean', 'euc_weighted_mean']]
df

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,14.183180,13.489501,11.680842,13.330479,13.330815,13.089284
1,15.188612,13.489501,12.564133,14.455599,14.455653,14.593701
2,16.367387,13.945114,14.427646,15.297827,15.298082,15.796754
3,17.602140,15.121991,14.267966,15.590489,15.591208,15.625839
4,14.964501,13.769848,13.358058,14.010092,14.010088,13.451506
5,16.136556,16.626208,14.756034,14.965697,14.965891,15.120731
6,14.533689,13.438194,11.893698,14.256403,14.255570,13.931088
7,16.813847,18.265522,15.809957,15.813477,15.813567,15.467608
8,15.182644,11.779689,13.637777,14.826320,14.825471,14.348341
9,16.945035,14.825623,13.348597,15.182108,15.182196,16.219168


In [186]:
df.iloc[[66]]

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
66,18.440514,15.800838,15.807024,15.86256,15.862276,15.588253


Comb dataframe contains the information of the gender and category of each runner. This will be used in a later notebook to test how accurate the predictions are in comparison to male, female, senior men, etc...

In [187]:
comb.join(df)

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,Senior Men (23-39),Male,14.183180,13.489501,11.680842,13.330479,13.330815,13.089284
1,Senior Men (23-39),Male,15.188612,13.489501,12.564133,14.455599,14.455653,14.593701
2,Senior Men (23-39),Male,16.367387,13.945114,14.427646,15.297827,15.298082,15.796754
3,Masters Men 1 (40-49),Male,17.602140,15.121991,14.267966,15.590489,15.591208,15.625839
4,Masters Men 1 (40-49),Male,14.964501,13.769848,13.358058,14.010092,14.010088,13.451506
5,Masters Men 1 (40-49),Male,16.136556,16.626208,14.756034,14.965697,14.965891,15.120731
6,Masters Men 1 (40-49),Male,14.533689,13.438194,11.893698,14.256403,14.255570,13.931088
7,Senior Men (23-39),Male,16.813847,18.265522,15.809957,15.813477,15.813567,15.467608
8,Senior Men (23-39),Male,15.182644,11.779689,13.637777,14.826320,14.825471,14.348341
9,Senior Men (23-39),Male,16.945035,14.825623,13.348597,15.182108,15.182196,16.219168


In [188]:
df = comb.join(df)

In [189]:
df.sort_values(by = 'weighted_mean')

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
798,Senior Men (23-39),Male,7.408082,7.012357,7.017497,7.886708,7.891352,7.896573
1213,Senior Men (23-39),Male,8.250002,8.106394,7.809304,7.940091,7.939923,7.997914
1571,Senior Men (23-39),Male,8.140912,8.141435,7.706042,7.963095,7.962934,7.950372
931,Senior Men (23-39),Male,7.761441,7.466542,7.346841,8.030584,8.030893,7.911646
334,Senior Men (23-39),Male,8.442793,8.572737,7.997654,8.091664,8.091770,8.031536
1572,Senior Men (23-39),Male,8.039194,8.141435,7.609757,8.119487,8.119373,8.113267
1072,Senior Men (23-39),Male,8.235962,8.056571,7.796014,8.127974,8.127964,8.127410
438,Masters Men 1 (40-49),Male,8.478720,8.607811,8.301849,8.145344,8.145315,8.147262
74,Senior Men (23-39),Male,8.051072,7.352225,7.745169,8.174848,8.174348,8.031906
1021,Masters Men 1 (40-49),Male,8.351205,8.125180,8.176995,8.193607,8.193431,8.195018


In [190]:
df.to_csv('../../data/pace_predictions.csv')

# Coverage

In [191]:
predict = pd.read_csv('../../data/pace_predictions.csv')

In [192]:
predict = len(predict)

In [193]:
(predict / first_df) * 100

98.4718826405868